In [1]:
# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy

In [2]:
import requests
from config import weather_api_key

In [3]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [4]:
lats = np.random.uniform(low=-90.000, high=90.000, size=500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [5]:
coordinates = list(lat_lngs)

In [6]:
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

37.55820898263856 -42.27556456507958
4.198594616437973 -41.916271168878694
-79.29015642610251 -69.25677584054999
-47.26683381146383 139.1259852488932
-81.4173766489436 -132.4022777362586
29.58594287291254 30.254947035774478
-69.44205664975817 -8.962790066303285
3.7625747243441197 23.830826600569083
-6.843537310209385 3.9462470186837777
-40.75999675605685 -67.0005597538211
26.394860588023477 -164.93760896111226
7.888173880472721 -120.9210160235682
-80.84910127433476 17.86052776711378
-31.537645979176602 23.584082252084983
49.308762363793704 -163.69445277096693
-47.88641755613591 14.21142843573
-62.51705640872324 -155.98772071502873
-71.73939884427718 52.659364040517914
86.52721000301668 42.320436360713956
71.28014762195261 47.56371963113557
19.74222545229439 124.8575133287041
-54.41667239593253 2.714850855046535
-57.674045607847276 -78.86756092983451
-48.30612230848829 4.865578821778399
-60.44432294966529 165.93666713519247
39.78617642831779 -159.02444383044462
-52.536150713726066 -9.53

In [7]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

259

In [13]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")


    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0ebe04528c163a168c1420eb41f3ce9e&q=ribeira+grande
-----------------------------
Data Retrieval Complete      
-----------------------------


In [9]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ribeira Grande,38.5167,-28.7000,62.78,84,36,3.00,PT,2022-10-17 02:16:22
1,Cururupu,-1.8283,-44.8683,76.86,84,4,4.85,BR,2022-10-17 02:18:00
2,Ushuaia,-54.8000,-68.3000,44.26,57,0,2.30,AR,2022-10-17 02:11:29
3,New Norfolk,-42.7826,147.0587,58.91,52,97,3.00,AU,2022-10-17 02:12:25
4,Rikitea,-23.1203,-134.9692,68.74,63,1,17.11,PF,2022-10-17 02:12:32
5,Cape Town,-33.9258,18.4232,63.12,92,75,14.97,ZA,2022-10-17 02:09:42
6,Bondo,3.8146,23.6866,66.43,98,94,0.92,CD,2022-10-17 02:18:01
7,Gamba,-2.6500,10.0000,74.93,92,92,3.74,GA,2022-10-17 02:17:40
8,General Roca,-39.0333,-67.5833,62.62,42,100,21.74,AR,2022-10-17 02:17:27
9,Kapaa,22.0752,-159.3190,84.18,67,0,10.36,US,2022-10-17 02:11:33


In [12]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")